<!-- (a) Warming up: Give the old school SQL query that produces three columns: ‘name’, ‘year’, and ‘rating’. -->
SELECT name, year, rating 
FROM "movies"."movie";

<!-- (b) Give the SQL/XML query that produces three columns, each with an element: <name>, <year>, <rating>. -->
SELECT 
    xmlforest(name AS "name", year AS "year", rating AS "rating") 
FROM movie;

<!-- (c) Give the SQL/XML query that produces a single column, with all three elements <name>, <year>, <rating>. -->
SELECT 
    xmlelement(NAME "movie", 
        xmlforest(name AS "name", year AS "year", rating AS "rating")
    ) 
FROM movie;

<!-- (d) Give the SQL/XML query that produces a single column, with all three elements <name>, <year>,
<rating> nested inside an element <movie>. (Bonus: add an attribute ’id’ with the movie identifier
‘mid’.) -->
SELECT 
    xmlelement(NAME "movie", 
        xmlattributes(mid AS "id"), 
        xmlforest(name AS "name", year AS "year", rating AS "rating")
    ) 
FROM movie;

<!-- (e) Write an ordinary SQL query that produces four columns, ‘name’, ‘year’, ‘rating’, and ’nr of roles’,
where the last column contains the number of actor roles for each movie.  -->
SELECT m.name, m.year, m.rating, COUNT(a.pid) AS "nr of roles"
FROM movie m
JOIN acts a ON m.mid = a.mid
GROUP BY m.mid;

<!-- Then change the SELECT
part of your query, such that it produces lines like the following, such that the result enumerates all
roles instead of counting them (and commit this too):
<movie><name>Jaws</name><year>1975</year><rating>8.2</rating><role>Chief Martin
Brody</role><role>Quint</role> ... </movie> -->
SELECT 
    xmlelement(NAME "movie",
        xmlelement(NAME "name", m.name),
        xmlelement(NAME "year", m.year),
        xmlelement(NAME "rating", m.rating),
        (SELECT xmlagg(xmlelement(NAME "role", a.role)) 
         FROM acts a 
         WHERE a.mid = m.mid)
    ) 
FROM movie m;

<!-- (f) Write an ordinary SQL query that produces four columns, ‘name’, ‘year’, ‘rating’, ’nr of roles’,
’nr or directors’ where the last column contains the number of directors for each movie. Tip: Solution
2e can be done with a GROUP BY clause, but also without.  -->
SELECT m.name, m.year, m.rating, 
       COUNT(DISTINCT a.pid) AS "nr of roles",
       COUNT(DISTINCT d.pid) AS "nr of directors"
FROM movie m
LEFT JOIN acts a ON m.mid = a.mid
LEFT JOIN directs d ON m.mid = d.mid
GROUP BY m.mid;

<!-- Then change the SELECT part of your
query, such that it produces lines like the following (and commit this too):
<movie><name>Jaws</name><year>1975</year><rating>8.2</rating><role>Chief Martin
Brody</role><role>Quint</role> ... <director>Steven Spielberg</director> </movie> -->
SELECT 
    xmlelement(NAME "movie",
        xmlelement(NAME "name", m.name),
        xmlelement(NAME "year", m.year),
        xmlelement(NAME "rating", m.rating),
        (SELECT xmlagg(xmlelement(NAME "role", a.role)) 
         FROM acts a 
         WHERE a.mid = m.mid),
        (SELECT xmlagg(xmlelement(NAME "director", p.name )) 
         FROM directs d, person p 
         WHERE d.mid = m.mid
         AND d.pid = p.pid)
    ) 
FROM movie m;